In [3]:
import meep as mp

# set data for wavelengt and n k materials
wavelengths = dataNK["Wavelength (nm)"].astype(float)
auN = dataNK["Au_n"].astype(float)
auK = dataNK["Au_k"].astype(float)
spiroN = dataNK["Spiro_n"].astype(float)
spiroK = dataNK["Spiro_k"].astype(float)
perovN = dataNK["Perov_n"].astype(float)
perovK = dataNK["Perov_k"].astype(float)
tio2N = dataNK["TiO2_n"].astype(float)
tio2K = dataNK["TiO2_k"].astype(float)


# 1um = 1000 nm
auHeight = 0.1  # 0.1 um or 100 nm
spiroHeight = 0.2  # 0.2 um or 200 nm
perovHeight = 0.37  # 0.37 um or 370 nm
tio2Height = 0.07  # 0.07 um or 70 nm
x_length = 1  # 1 um or 1000 nm
y_length = 1  # 1 um or 1000 nm


# Define the refractive index of the materials
au = [mp.Medium(index=complex(n, k)) for n, k in zip(auN, auK)]
spiro = [mp.Medium(index=complex(n, k)) for n, k in zip(spiroN, spiroK)]
perov = [mp.Medium(index=complex(n, k)) for n, k in zip(perovN, perovK)]
tio2 = [mp.Medium(index=complex(n, k)) for n, k in zip(tio2N, tio2K)]


# Define the geometry of the simulation
def geometryNK(index: int) -> List[mp.Block]:
    geometry = [
        mp.Block(
            size=mp.Vector3(x_length, y_length, auHeight),
            material=au[index],
            center=mp.Vector3(z=-auHeight / 2),
        ),
        mp.Block(
            size=mp.Vector3(x_length, y_length, spiroHeight),
            material=spiro[index],
            center=mp.Vector3(z=-auHeight - spiroHeight / 2),
        ),
        mp.Block(
            size=mp.Vector3(x_length, y_length, perovHeight),
            material=perov[index],
            center=mp.Vector3(z=-auHeight - spiroHeight - perovHeight / 2),
        ),
        mp.Block(
            size=mp.Vector3(x_length, y_length, tio2Height),
            material=tio2[index],
            center=mp.Vector3(z=-auHeight - spiroHeight - perovHeight - tio2Height / 2),
        ),
    ]

    return geometry


print(geometryNK(0))

ModuleNotFoundError: No module named 'meep'